In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
dbutils.widgets.text("StartDate", "")

StartDate = dbutils.widgets.get("StartDate")
print(f"StartDate: {StartDate}")

StartDate: 20250831


In [0]:
import datetime

date = datetime.datetime.strptime(str(StartDate), "%Y%m%d")

year_val  = str(date.year)
month_val = str(f"{date.month:02}")
day_val   = str(f"{date.day:02}")

print(f"Processing Date  Year: {year_val}, Month: {month_val}, Day: {day_val}")

Processing Date  Year: 2025, Month: 08, Day: 31


In [0]:
# Function to merge data into Delta table
def merge_delta(df, target_table, merge_keys):
    """
    Merge a DataFrame into a Delta table based on given keys.

    Args:
        df (DataFrame): The source DataFrame (new data).
        target_table (str): The full target table name (catalog.schema.table).
        merge_keys (list): List of column names to use as merge keys.
    """
    delta_table = DeltaTable.forName(spark, target_table)
    
    # Build merge condition dynamically
    merge_condition = " AND ".join([f"dt_table.{k} = new_record_df.{k}" for k in merge_keys])
    
    delta_table.alias("dt_table")\
        .merge(df.alias("new_record_df"),merge_condition)\
        .whenMatchedDelete() \
        .whenNotMatchedInsertAll()\
        .execute()
    print(f"Merge completed for {target_table} on keys {merge_keys}")

In [0]:

# Create sales fact table from silver data
def create_sales_fact_gold(df):
    
    sales_fact_df = df.groupBy(
        "product_id", 
        "product_name", 
        "category",
        year("order_date").alias("sales_year"),
        month("order_date").alias("sales_month")
    ).agg(
        countDistinct("order_id").alias("total_orders"),
        sum("quantity").alias("quantity_sold"),
        sum(col("price") * col("quantity")).alias("gross_sales"),
        sum((col("price") * col("quantity")) - col("discount")).alias("net_revenue")
    )
    
    return sales_fact_df


# Filter only the relevant Silver partition
gold_df = (
    spark.read.table("ecommerce_catalog.silver.SalesCleansedData")
    .filter((col("year") == year_val) & (col("month") == month_val) & (col("day") == day_val))
)
sales_fact_df = create_sales_fact_gold(gold_df)\
    .orderBy("product_id", "sales_year", "sales_month")

display(sales_fact_df)

# sales_fact_df.createOrReplaceTempView("sales_fact")

target_table = "ecommerce_catalog.gold.sales_fact"

# Write to gold layer
if not spark.catalog.tableExists(target_table):
    print("Initializing the full load...")
    (
        sales_fact_df.write.format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(target_table)
    )
else:
    print("Running incremental merge...")
    merge_delta(sales_fact_df, target_table, ["product_id", "sales_year", "sales_month"])

product_id,product_name,category,sales_year,sales_month,total_orders,quantity_sold,gross_sales,net_revenue
P101,iPhone 13,Electronics,1900,1,3,6,4800.0,4775.0
P101,iPhone 13,Electronics,2023,1,1,4,3200.0,3170.0
P101,iPhone 13,Electronics,2023,3,1,3,2400.0,2400.0
P101,iPhone 13,Electronics,2023,9,1,2,1600.0,1595.0
P101,iPhone 13,Electronics,2023,10,1,3,2400.0,2350.0
P101,iPhone 13,Electronics,2024,2,1,5,4000.0,3990.0
P101,iPhone 13,Electronics,2024,3,2,4,3200.0,3180.0
P101,iPhone 13,Electronics,2024,6,1,2,1600.0,1550.0
P101,iPhone 13,Electronics,2024,8,1,2,1600.0,1595.0
P101,iPhone 13,Electronics,2024,9,1,5,4000.0,4000.0


Running incremental merge...
Merge completed for ecommerce_catalog.gold.sales_fact on keys ['product_id', 'sales_year', 'sales_month']


In [0]:

# Create aggregated insights in gold layer
def create_aggregated_insights(df):
    
    # 1. Total revenue per category
    revenue_by_category = (df.groupBy("category")
        .agg(sum("net_revenue").alias("total_revenue"))
    ).orderBy(desc("total_revenue"))
    
    # 2. Top 5 products by revenue
    top_products =  (df.groupBy("product_id", "product_name")
    .agg(sum("net_revenue").alias("total_revenue"))
    .orderBy(col("total_revenue").desc())
    .limit(5)
)
    
    # 3. Monthly sales trends
    monthly_trends = (sales_fact_df.groupBy("category", "sales_year", "sales_month")
    .agg(sum("net_revenue").alias("monthly_revenue"))
    .orderBy("sales_year", "sales_month")
)
   
    return revenue_by_category, top_products, monthly_trends


# Read from the gold sales_fact table
src_df = spark.read.table(target_table)

revenue_by_category, top_products, monthly_trends = create_aggregated_insights(src_df)

# Save results back into gold layer
revenue_by_category.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("ecommerce_catalog.gold.revenue_by_category")
top_products.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("ecommerce_catalog.gold.top_products")
monthly_trends.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("ecommerce_catalog.gold.monthly_trends")

print("Gold layer tables refreshed successfully.")


Gold layer tables refreshed successfully.


In [0]:
%sql
SELECT 
    category, 
    SUM(net_revenue) AS total_revenue
FROM ecommerce_catalog.gold.sales_fact
GROUP BY category
ORDER BY total_revenue DESC;

category,total_revenue
Electronics,131634.0
Home,80026.0
Clothing,17116.0
UNKNOWN,5226.0
Books,4160.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    product_id, product_name, 
    SUM(net_revenue) AS product_revenue
FROM ecommerce_catalog.gold.sales_fact
GROUP BY product_id, product_name
ORDER BY product_revenue DESC
LIMIT 5;

product_id,product_name,product_revenue
P102,Samsung Galaxy,57725.0
P302,Dining Table,40055.0
P101,iPhone 13,39745.0
P301,Sofa,32165.0
P104,Apple Watch,22730.0


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    category,
    sales_year,
    sales_month,
    SUM(net_revenue) AS monthly_revenue,
    SUM(quantity_sold) AS monthly_quantity,
    SUM(total_orders) AS monthly_orders
FROM ecommerce_catalog.gold.sales_fact
GROUP BY category, sales_year, sales_month
ORDER BY sales_year, sales_month, category DESC;


category,sales_year,sales_month,monthly_revenue,monthly_quantity,monthly_orders
Home,1900,1,830.0,4,3
Clothing,1900,1,40.0,2,1
Books,1900,1,655.0,16,6
Electronics,2023,1,11120.0,28,8
Clothing,2023,1,725.0,21,7
Books,2023,1,205.0,14,4
UNKNOWN,2023,2,1985.0,6,2
Home,2023,2,1795.0,4,1
Electronics,2023,2,3220.0,10,3
Clothing,2023,2,845.0,16,4


Databricks visualization. Run in Databricks to view.